
# PNP Finite Difference

The equations:



$$\frac{\partial C_p}{\partial t} = \nabla^2 C_p + \nabla(C_p\nabla \phi)$$

$$\frac{\partial C_n}{\partial t} = \nabla^2 C_n - \nabla(C_n\nabla \phi)$$

$$\nabla^2 \phi = C_n - C_p$$


And the boundary conditions:
$$\nabla C_p (x=L_x) = \nabla C_p (x=0) = \nabla C_n (x=L_x) = \nabla C_n (x=0)=0$$

$$\phi(L_x) = 1, \phi(0) = -1$$

In [37]:
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

# Simulation Constants
L = 100
M = 1000
dx = L / M  

T = 5

N = 1000
dt = T / N

neumann = True


## Finite Difference

### Finding $\phi$

I will use the implicit finite difference method.
After applying finite difference, the potential $\phi(x,t)$ at the point $x_m, 0<m<M-1$ would be:

$$\frac{\Delta x^2}{2}(p_m - n_m) = \phi_m -\frac{1}{2} \phi_{m+1} -\frac{1}{2} \phi_{m-1} $$

#### Dirichlet
So solving for $\phi$ at a certain time would mean solving the following matrix:

$$
\begin{gather}
   \begin{bmatrix}
      1 & -\frac{1}{2} & 0 & 0 & ... \\
      -\frac{1}{2} & 1 & -\frac{1}{2} & 0 & ... \\
      ... & ... \\
      ... & 0 & -\frac{1}{2} & 1 & -\frac{1}{2} \\
      ... & 0 & 0 & -\frac{1}{2} & 1
   \end{bmatrix}
   \begin{bmatrix}
      \phi_1 \\
      \phi_2 \\
      ... \\
      \phi_{M-3} \\
      \phi_{M-2}
   \end{bmatrix}
 
 =
 \frac{\Delta x^2}{2}
  \begin{bmatrix}
   p_1 - n_1 + \frac{1}{\Delta x^2} \phi_0 \\
   p_2 - n_2 \\
   ... \\
   p_{M-3} - n_{M-3} \\
   p_{M-2} - n_{M-2} + \frac{1}{\Delta x^2} \phi_M

   \end{bmatrix}
\end{gather} $$

Notice that A is composed of $(M-2)$ ones, and $(M-3)$ elements $-\frac{1}{2}$ on each diagonal.  

#### Neumann
With Neumann boundary conditions we get for the edges:

$$ \frac{1}{2}\phi_1 - \frac{1}{2}\phi_2 = \frac{\Delta x (\Delta x - n_l)}{2} $$

$$ \frac{1}{2}\phi_{M-2} - \frac{1}{2}\phi_{M-3} = \frac{\Delta x (\Delta x - n_r)}{2} $$

Where $n_l, n_r$ are the fluxes at the edges.


In [38]:
def get_phi(phi_0, phi_M, n, p, dx, M=M):

    # TODO: might be a problem with the loops and it should start at 1 instead of 0.
    # Creating the coeff matrix
    line_coeff = []
    
    coeff_matrix = np.diag(np.ones(M-2)) + np.diag(-0.5*np.ones(M-3), 1) + np.diag(-0.5*np.ones(M-3), -1)
    # print(coeff_matrix.shape, coeff_matrix)

    # for m in range(M-2):
    #     coeff = np.zeros(M-2)
    #     coeff[m] = 1

    #     if m == 0:
    #         coeff[m+1] = -1/2
        
    #     elif m == M-3:
    #         coeff[m-1] = -1/2
    #     else:
    #         coeff[m+1] = -1/2
    #         coeff[m-1] = -1/2
    
    #     line_coeff.append(coeff)

    # coeff_matrix = np.vstack(line_coeff)

    # Creating the RHS result vector
    result_vector = np.array([p[m]-n[m] for m in range(2, M-2)])

    right_res = p[M-2] - n[M-2] + phi_M/(dx**2)
    left_res = p[1] - n[1] + phi_0/(dx**2)

    result_vector = ((dx**2)/2)*np.append(np.insert(result_vector, 0, left_res), right_res)
    # print(result_vector.shape, result_vector)
    
    # line_result = []
    # for m in range(1, M-1):
    #     result = ((dx**2)/2)*(p[m]-n[m])

    #     if m == 0:
    #         result += phi_0/2

    #     elif m == M-2:
    #         result += phi_M/2
        
    #     line_result.append(result)

    
    # result_vector = np.vstack(line_result)

    # Solving for phi and adding the boundry

    phi_trimmed = np.linalg.solve(coeff_matrix, result_vector)
    phi = np.append(np.insert(phi_trimmed, 0, phi_0), phi_M)

    return phi

# Test
# get_phi(-1, 1, np.random.rand(M), np.random.rand(M), dx)


## Iterating $p, n$

Iterating $p(x,t), n(x,t)$ means finding $p_m^{i+1}, n_m^{i+1}$. We assume that we have $p_m^j, n_m^j$ for every $m, j<i$.
I will use explicit central difference method to get:

$$ 
\begin{align}
p_m^{i+1} = p_m^{i} + \frac{\Delta t}{\Delta x^2} \left( p_{m+1}^i + p_{m-1}^i - 2p_m^i + \frac{1}{4}(p_{m+1}^i-p_{m-1}^i)(\phi_{m+1}^i-\phi_{m-1}^i) + p_m^i(\phi_{m+1}^i+\phi_{m-1}^i-2\phi_m^i) \right)
\end {align}
$$

$$ 
\begin{align}
n_m^{i+1} = n_m^{i} + \frac{ \Delta t}{\Delta x^2} \left( n_{m+1}^i + n_{m-1}^i - 2n_m^i - \frac{1}{4}(n_{m+1}^i-n_{m-1}^i)(\phi_{m+1}^i-\phi_{m-1}^i) - n_m^i(\phi_{m+1}^i+\phi_{m-1}^i-2\phi_m^i) \right)
\end {align}
$$


In [39]:
def iterate_concentration(c, phi, dx, dt, positive, M=M):
    """
    Assume c and phi are 1XM vectors 
    c_pp - c-prime-prime, cp - c-prime, phi_pp - phi-prime-prime, phi_p - phi-prime
    
    """
    sign = 1 if positive else -1

    c_1, c_M = c[0], c[M-1]
    phi_1, phi_M = phi[0], phi[M-1]
    # c_prev_1, c_prev_M = c_prev[0], c_prev[M-1]


    c_rolled = np.roll(c, 1)
    c_rolled_back = np.roll(c, -1)
    np.put(c_rolled, [0, -1], [c_1, c_M]) # Creating c_{m+1}
    np.put(c_rolled_back, [0, -1], [c_1, c_M]) # Creating c_{m-1}

    phi_rolled = np.roll(phi, 1)
    phi_rolled_back = np.roll(phi, -1)
    np.put(phi_rolled, [0, -1], [phi_1, phi_M]) # Creating phi_{m+1}
    np.put(phi_rolled_back, [0, -1], [phi_1, phi_M]) # Creating phi_{m+1}

    c_pp = c_rolled + c_rolled_back - 2*c
    c_p = c_rolled - c
    
    phi_pp = phi_rolled + phi_rolled_back - 2*phi
    phi_p = phi_rolled - phi
    # print(phi_p, phi_pp)

    c_iter = c + dt/(dx**2)*(c_pp + sign*c_p*phi_p + sign*c*phi_pp)
    # print(c-c_iter)

    return c_iter

# Test
# iterate_concentration(np.random.rand(M), np.random.rand(M), np.random.rand(M), 0.1, 0.001, True)


## Simulation
We iterate according to the following algorithm:

Assume we know $ \phi_m^{i-1} n_m^{i-1}, p_m^{i-1}$ for every $m = 1,2,...M$ (and the boundry conditions)

Then iterate to the next step according to:

1. Finding $n_m^{i}, p_m^{i}$ for every $m$ based on $\phi_m^{i-1}, n_m^{i-1}, p_m^{i-1}$
2. Finding $\phi_m^{i}$ based on $n_m^{i}, p_m^{i}$ for every $m$, using implicit finite difference. 

In [40]:
def iterate(phi, n, p, dx, dt):
    phi_0 = phi[0]
    phi_M = phi[M-1]

    next_n = iterate_concentration(n, phi, dx, dt, positive=False)
    next_p = iterate_concentration(p, phi, dx, dt, positive=True)
    next_phi = get_phi(phi_0, phi_M, next_n, next_p, dx)

    return next_phi, next_n, next_p

In [41]:

# Physics Constants
D = 1 
epsilon = 1

x = np.linspace(0, L, M)
t = np.linspace(0, N, T)
n, p, phi = np.zeros((N, M)), np.zeros((N, M)), np.zeros((N, M))

# Initial Conditions
n[0, :] = 0.5*(1+0.1*np.random.rand(M))
p[0, :] = 0.5*(1+0.1*np.random.rand(M))
phi[0, :] = 1/(1+np.exp(-(x-L/2)))


# Dirichlet Boundry
phi[:, 0] = -1
phi[:, M-1] = 1

n[:, 0] = 0
n[:, M-1] = 2
p[:, 0] = 2
p[:, M-1] = 0

In [42]:
for i in range(1, N):
    # print(phi[i,:])


    phi_i, n_i, p_i = iterate(phi[i-1, :], n[i-1, :], p[i-1, :], dx, dt)
    

    n[i, :] = n_i
    p[i, :] = p_i
    phi[i, :] = phi_i


    plt.plot(x, p[i-1,:], label="$C_p$" )
    plt.plot(x, n[i-1,:], label="$C_n$")
    plt.legend()

    plt.pause(0.01)
    display.clear_output(wait=True)

    if i == N-1:
        phi[i, :] = get_phi(phi[i, 0], phi[i, M-1], p[i, :], n[i, :], dx)


KeyboardInterrupt: 